In [2]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import random

# Set random seed for reproducibility
random.seed(42)

# Set display options to show all columns
pd.set_option('display.max_columns', None)

# Define base directory
base_directory = os.path.abspath('../..')

# Read and clean US Census data
csv_file_path = os.path.join(base_directory, '_input_raw_data', 'social_explorer_tract_ancestry_data', 'R13669333_SL140.csv')
ancestry_df = pd.read_csv(csv_file_path)

# Drop the 0th row and reindex
ancestry_df = ancestry_df.drop(ancestry_df.index[0]).reset_index(drop=True)

# Filter for columns before and including "Census Tract", and columns including and after "Total Population"
start_col = "Census Tract"
end_col = "Total Population"

# Get column indices
start_idx = ancestry_df.columns.get_loc(start_col)
end_idx = ancestry_df.columns.get_loc(end_col)

# Select the columns
selected_columns = ancestry_df.columns[:start_idx + 1].tolist() + ancestry_df.columns[end_idx:].tolist()
filtered_df = ancestry_df[selected_columns]

# Convert relevant columns to numeric
numeric_columns = filtered_df.columns[filtered_df.columns.get_loc("Total Population:") + 1:]
filtered_df[numeric_columns] = filtered_df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Read shapefile data
shapefile_path = os.path.join(base_directory, '_input_raw_data', 'us_census_cartography', 'cb_2022_us_tract_500k')
shapefile = gpd.read_file(shapefile_path)

# Merge geospatial data and US Census data
filtered_df['FIPS'] = pd.to_numeric(filtered_df['FIPS'], errors='coerce').astype(np.int64)
shapefile['GEOID'] = pd.to_numeric(shapefile['GEOID'], errors='coerce').astype(np.int64)
merged_df = pd.merge(filtered_df, shapefile, left_on='FIPS', right_on='GEOID', how='inner')

# Create GeoDataFrame
gdf = gpd.GeoDataFrame(merged_df, geometry='geometry')

# List of columns for "People Reporting Multiple Ancestries"
multiple_ancestries_columns = [

"People Reporting Multiple Ancestries: Sub-Saharan African: Cape Verdean",
"People Reporting Multiple Ancestries: Sub-Saharan African: Ethiopian",
"People Reporting Multiple Ancestries: Sub-Saharan African: Ghanaian",
"People Reporting Multiple Ancestries: Sub-Saharan African: Kenyan",
"People Reporting Multiple Ancestries: Sub-Saharan African: Liberian",
"People Reporting Multiple Ancestries: Sub-Saharan African: Nigerian",
"People Reporting Multiple Ancestries: Sub-Saharan African: Senegalese",
"People Reporting Multiple Ancestries: Sub-Saharan African: Sierra Leonean",
"People Reporting Multiple Ancestries: Sub-Saharan African: Somali",
"People Reporting Multiple Ancestries: Sub-Saharan African: South African",
"People Reporting Multiple Ancestries: Sub-Saharan African: Sudanese",
"People Reporting Multiple Ancestries: Sub-Saharan African: Ugandan",
"People Reporting Multiple Ancestries: Sub-Saharan African: Zimbabwean",
"People Reporting Multiple Ancestries: Sub-Saharan African: African",
"People Reporting Multiple Ancestries: Sub-Saharan African: Other Sub-Saharan African",
]

# List to store the dot data
dots_data = []

# Function to generate random points within a polygon
def generate_random_points(polygon, num_points):
    points = []
    minx, miny, maxx, maxy = polygon.bounds
    while len(points) < num_points:
        random_point = gpd.points_from_xy([random.uniform(minx, maxx)], [random.uniform(miny, maxy)])[0]
        if polygon.contains(random_point):
            points.append(random_point)
    return points

# Generate dots for each column and store in dots_data
for column in multiple_ancestries_columns:
    for _, row in gdf.iterrows():
        try:
            num_dots = int(float(row[column]) / 1)  # 1 dot per 100 people
            if num_dots > 0 and not row['geometry'].is_empty:
                random_points = generate_random_points(row['geometry'], num_dots)
                for point in random_points:
                    dots_data.append({'column': column, 'latitude': point.y, 'longitude': point.x})
        except ValueError:
            continue

# Create a DataFrame from dots_data
dots_df = pd.DataFrame(dots_data)

# Export to CSV for Tableau
output_file_path = os.path.join(base_directory, 'dots_data_for_sub_saharan_africa.csv')
dots_df.to_csv(output_file_path, index=False)

print(f"Data exported to {output_file_path}")


C:\Users\cjong\AppData\Local\Temp\ipykernel_6424\2403387798.py:18: DtypeWarning: Columns (0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241) have mixed types. Specify dtype option on import or set lo

Data exported to c:\Users\cjong\Projects\geospatial\ancestry_dot_map\dots_data_for_sub_saharan_africa.csv
